In [59]:
import numpy as np
import pandas as pd
import requests
import time

### API 구성요소

### TFT-League-V1

Rank league id 데이터 입니다.

TFT rank는 아이언, 브론즈, 실버, 골드, 플레티넘, 다이아몬드, 마스터, 그랜드마스터, 챌린저 순으로 총 9개의 rank로 구성되어 있고, 아이언부터 다이아몬드 랭크까지 각각 4개의 등급이 따로 존재합니다.(ex : ,,, 실버1, 골드4, 골드3, 골드2, 골드1, 플레티넘4, ...)


#### 1. /tft/league/v1/challenger

챌린저 랭크 유저 데이터입니다. 챌린저는 300명으로 구성되어 있으며, 최상위 유저입니다. 

league 이름과 id, tier, 유저로 구성되어 있으며 유저에 대한 정보는 entries에 list로 존재합니다.

entries에서 나타나는 유저들의 정보는 다음과 같습니다.

1) summonerid : 소환사 id code입니다.

2) summonerName : 게임에서 사용하고 있는 유저의 id입니다.

3) leaguePoints : 현재 rank 점수입니다.

4) rank : rank입니다. 챌린저이므로 모든 유저가 'I'입니다.

5) wins : 이번 rank 측정 기간 중 승리 수입니다.

6) losses : 이번 rank 측정 기간 중 패배 수입니다.

7) veteran : 베테랑 유무입니다. (이번 시즌 게임 수가 100회 이상이면 베테랑)

8) inactive : 최근 게임 참여 적극성

9) freshBlood : 이번 시즌이 첫 시즌인 경우 T

10) hotStreak : 3연승 이상 중


#### 2. /tft/league/v1/entries/{tier}/{division}


각 rank에 따른 유저 정보입니다.

아이언4부터 다이아몬드1 중 원하는 티어를 선택하면, 그 티어에 위치하는 유저 정보를 알 수 있습니다.

제공되는 정보는 다음과 같습니다.

1) leagueId : 현재 소속된 리그 id

5) summonerid : 소환사 id code입니다.

6) summonerName : 게임에서 사용하고 있는 유저의 id입니다.


2) queueType : 게임 타입

3) tier : rank tier입니다.

4) rank : rank입니다. 챌린저이므로 모든 유저가 'I'입니다.(1~4를 표시)


7) leaguePoints : 현재 rank 점수입니다.


8) wins : 이번 rank 측정 기간 중 승리 수입니다.

9) losses : 이번 rank 측정 기간 중 패배 수입니다.

10) veteran : 베테랑 유무입니다. (이번 시즌 게임 수가 100회 이상이면 베테랑)

11) inactive : 최근 게임 참여 적극성

12) freshBlood : 이번 시즌이 첫 시즌인 경우 T

13) hotStreak : 3연승 이상 중


IV, III

페이지마다 205명의 유저 정보가 존재 but, page가 바뀔 때 이전 page의 후반 5명 정보부터 시작함(한 page당 200명 정보 얻을 수 있음)




In [95]:
class load_data:
    
    def __init__(self, api_key):
        
        self.api_key = api_key
        self.request_header  = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/97.0.4692.99 Safari/537.36",
    "Accept-Language": "ko-KR,ko;q=0.9,en-US;q=0.8,en;q=0.7",
    "Accept-Charset": "application/x-www-form-urlencoded; charset=UTF-8",
    "Origin": "https://developer.riotgames.com",
    "X-Riot-Token": self.api_key
    }
        self.base_url = "https://kr.api.riotgames.com/tft/"
    
   
    
    
    
    def extract_sky(self, tier):
        
        if tier not in ['grandmaster', 'challenger', 'master']:
            
            print('해당 함수에서는 master, grandmaster, challenger만 기술할 수 있습니다.')
            
            return None
        
        code_name = f"league/v1/{tier}"
        account_id=requests.get(f"{self.base_url}{code_name}", headers=self.request_header).json()
        
        return pd.DataFrame(account_id['entries'])
    

    
    
    def extract_other(self, tier, level, num_page):
        
        
        
        
        
        code_name = f"league/v1/entries/{tier}/{level}"
        
        result =  pd.DataFrame((requests.get(f"{self.base_url}{code_name}?page=1", headers = self.request_header).json()))
        
        
        
        
        if num_page!=1:
            
        
            for page in range(2, num_page+1):

                account_id = requests.get(f"{self.base_url}{code_name}?page={page}", headers = self.request_header).json()
                account_id = pd.DataFrame(account_id)
                
                if account_id.shape[0]<=5:
                    return result
                    
                else:
                    account_id = account_id.iloc[5:]
                
                result = pd.concat([result, account_id])
                
                time.sleep(2)
        

        return result
    
    
    def extract_game_by_summoner(self, id_name, world, game_count):
        
        id_code_name = f"summoner/v1/summoners/by-name/{id_name}"

        
        puu_id = requests.get(f"{self.base_url}{id_code_name}", headers=self.request_header).json()['puuid']
        
        
        game_url = f"https://{world}.api.riotgames.com/tft/"
        
        game_code_name = f"match/v1/matches/by-puuid/{puu_id}/ids?count={game_count}"
        
        game_result = requests.get(f"{game_url}{game_code_name}", headers=self.request_header).json()
        
        return game_result
        
    
    
    def extract_game_result(self, game_id, world):
        
        game_url = f"https://{world}.api.riotgames.com/tft/"
        
        game_result_code_name = f"match/v1/matches/{game_id}"
        
        result = requests.get(f"{game_url}{game_result_code_name}", headers=self.request_header).json()
        
        return result
        
        
    